# 第四节 链（chain）,输出解析器与记忆(memory)

In [2]:
#导入语言模型
import os
from langchain_community.llms import Tongyi
from langchain_community.llms import SparkLLM
from langchain_community.llms import QianfanLLMEndpoint

import pandas as pd
#导入模版
from langchain.prompts import PromptTemplate

#导入聊天模型
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain_community.chat_models import ChatSparkLLM
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_community.chat_models import QianfanChatEndpoint

#输入三个模型各自的key

os.environ["DASHSCOPE_API_KEY"] = ""

os.environ["IFLYTEK_SPARK_APP_ID"] = ""
os.environ["IFLYTEK_SPARK_API_KEY"] = ""
os.environ["IFLYTEK_SPARK_API_SECRET"] = ""

os.environ["QIANFAN_AK"] = ""
os.environ["QIANFAN_SK"] = ""

In [19]:
model_ty = Tongyi(temperature=0.1)

## 链(chain)

只使用一个 LLM 去开发应用，比如聊天机器人是很简单的，但更多的时候，我们需要用到许多 LLM 去共同完成一个任务，这样原来的模式就不足以支撑这种复杂的应用。

为此 LangChain 提出了 Chain 这个概念，也就是一个所有组件的序列，能够把一个个独立的 LLM 链接成一个组件，从而可以完成更复杂的任务。举个例子，我们可以创建一个 chain，用于接收用户的输入，然后使用提示词模板将其格式化，最后将格式化的结果输出到一个 LLM。通过这种链式的组合，就可以构成更多更复杂的 chain。

在 LangChain 中有许多实现好的 chain，以最基础的 LLMChain 为例，它主要实现的就是接收一个提示词模板，然后对用户输入进行格式化，然后输入到一个 LLM，最终返回 LLM 的输出。

In [27]:
template = """我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？"""

prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)
model_ty.invoke(prompt.format(product="彩色袜子"))

'当然可以，以下是我为您的彩色袜子公司提出的五个中文名称建议：\n\n1. 彩韵袜业：强调袜子的色彩丰富和韵律感，同时也暗示了产品的艺术性和时尚感。\n2. 明彩足下：寓意穿上这些袜子能让人的脚步都明亮多彩，生活因此增添乐趣。\n3. 色舞袜坊：形象地描绘出袜子色彩斑斓，仿佛在脚下跳舞的情景，富有活力和动感。\n4. 七彩织梦：七彩代表各种颜色，织梦则象征着用色彩编织美好的梦想，富有诗意。\n5. 彩悦足尖：强调袜子带给人们的快乐和满足感，同时“悦”字也传达了高品质的使用体验。\n\n希望这些建议能对您有所帮助！'

In [28]:
from langchain.chains import LLMChain
 
prompt = PromptTemplate(
    input_variables=["product"],
    template="我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？",
)

chain = LLMChain(llm=model_ty, prompt=prompt)

In [29]:
chain.invoke("彩色袜子")

{'product': '彩色袜子',
 'text': '1. 彩韵袜业：这个名字融合了“色彩”和“韵律”，寓意袜子的多彩和时尚，同时传达出一种活泼、灵动的品牌形象。\n\n2. 明彩足下：强调袜子的鲜艳色彩，同时“明”字也寓意产品质量明亮、清晰，给人以良好的视觉感受。\n\n3. 七彩足迹：以七彩代表丰富的颜色选择，"足迹"则暗示袜子与人们的日常生活紧密相连，每一步都充满色彩。\n\n4. 颜舞袜艺： "颜舞"寓意色彩的跳跃和生动，"袜艺"则强调制作工艺的艺术性和专业性。\n\n5. 色彩织梦：通过“色彩”和“织梦”结合，表达出产品能为人们的生活增添色彩，如同编织美好的梦想一般。'}

### 多个提示词

In [30]:
# 也可以使用generate，apply的生成结果简洁点
input_list = [
    {"product": "袜子"},
    {"product": "电脑"},
    {"product": "男鞋"}
]

chain.apply(input_list)

[{'text': '当然可以，以下是我为您的袜子公司提出的五个中文名字建议：\n\n1. "足悦"：这个名字寓含了袜子带给人们的舒适和快乐，同时也表达了对顾客双脚的呵护。\n2. "行云袜业"：行云象征流畅、自由，暗示袜子的柔软和无束缚感，同时也给人以轻松愉快的联想。\n3. "舒步"：强调穿着的舒适感和行走的自在，适合注重品质的袜子品牌。\n4. "纤丝韵"：纤丝代表袜子的细腻材质，韵则传达出一种雅致和时尚的氛围。\n5. "足尚工坊"：将“足”与“时尚”结合，表示公司的产品既实用又潮流，工坊则带有一种手工艺的温馨感。\n\n希望这些建议能对您有所帮助！如果您需要更多选择或者有特定的要求，欢迎进一步告诉我。'},
 {'text': '当然可以，以下是我为您的电脑生产公司提出的五个可能的中文名字建议：\n\n1. "智域科技"：这个名字突出了智能和科技的元素，同时也传达出公司在科技领域的专业和广阔视野。\n2. "灵思电脑"："灵思"寓意创新思维和灵活应用，强调产品的智能化和人性化设计。\n3. "锐志数码"："锐志"表示公司具有坚定的进取心和敏锐的市场洞察力，"数码"则直接指代产品类别。\n4. "翔云计算机"："翔云"象征着自由、高效，寓意公司的产品能帮助用户如同在云端般轻松工作。\n5. "博睿创研"："博睿"意味着广博的知识和智慧，"创研"则强调了公司的研发创新精神。\n\n希望这些建议能对您有所帮助！在选择名字时，您还可以考虑公司的品牌定位、目标市场以及想要传达的品牌形象等因素。'},
 {'text': '1. 雄步天下：这个名字寓含了男士的步伐稳健，有领导力，如同踏遍天下，寓意鞋子的质量和舒适度能让消费者自信前行。\n2. 风度翩翩：风度翩翩常用来形容男士的优雅气质，以此为名，可以强调产品的时尚感和品质感，适合注重风格的男士。\n3. 铠甲足下：铠甲象征保护和力量，暗示鞋子能为双脚提供良好的保护，适合强调鞋的耐用性和舒适性。\n4. 立绅之履：立绅寓意站立的绅士，代表了优雅、稳重，体现出男鞋的高端和品质，同时也暗示穿着者有良好的品格。\n5. 品逸先生：品逸代表品味和超越，强调产品的独特设计和卓越品质，适合追求个性化和高品质生活的男士。'}]

### 多个变量

In [35]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="我希望你能充当新公司的命名顾问。公司新推出一款{product}产品，请帮给新产品取一个{style}中文名字，给出五个选项？",
)

chain_m = LLMChain(llm=model_ty, prompt=prompt)

In [36]:
chain_m.invoke({"product": "混动越野汽车","style": "科技感"})

{'product': '混动越野汽车',
 'style': '科技感',
 'text': '当然可以，以下是我为您的混动越野汽车产品提出的五个具有科技感的中文名字选项：\n\n1. "凌云智行"：这个名字融合了凌云壮志的意象和智能行驶的概念，展现出汽车的高性能和科技特性。\n\n2. "电掣星驰"：电掣代表混合动力的速度感，星驰则寓意科技如星辰般引领前行，富有未来感。\n\n3. "绿动未来"：突出了环保的混动特点，同时“未来”一词暗示了科技的前瞻性。\n\n4. "影豹锐动"：影豹象征速度和力量，锐动强调了汽车的科技敏锐性和动感。\n\n5. "智越星际"：智越代表智能超越，星际则象征着探索未知的科技精神，适合一款高科技的越野车。\n\n希望这些建议能对您有所帮助！如果您需要更多的选择或者有其他特定的要求，欢迎告诉我。'}

### 聊天模型的链


In [39]:
chat_ty = ChatTongyi()

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="我希望你能充当新公司的命名顾问。一个生产{product}的公司好的中文名字是什么,请给出五个选项？?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chain_chat = LLMChain(llm=chat_ty, prompt=chat_prompt_template)

In [40]:
chain_chat.invoke({"product": "彩色袜子"})

{'product': '彩色袜子',
 'text': '当然可以，为您的彩色袜子公司起名是一项创意任务。以下是五个富有创意且适合的中文名字建议：\n\n1. 彩云袜业（Cǎi Yún Wà Yè）："彩云"象征多彩和轻盈，与袜子的色彩丰富相呼应。\n2. 花漾织语（Huā Yàng Zhī Yǔ）："花漾"形容色彩斑斓，"织语"暗示手工编织，温馨而女性化。\n3. 七彩丝履（Qī Cǎi Sī Lǚ）："七彩"直接表达颜色多样，"丝履"显得优雅且与袜子相关。\n4. 绚彩踏浪（Xuàn Cǎi Tà Làng）："绚彩"代表丰富的色彩，"踏浪"有轻松行走的意象，适合运动或时尚定位。\n5. 色彩足迹（Sè Cǎi Zú Jì）：简洁明了，强调色彩的同时，"足迹"让人联想到穿着袜子的生活场景。\n\n每个名字都有其独特含义和感觉，您可以根据公司的品牌形象、目标市场和个人喜好选择最适合的一个。'}

classwork 1

1, 创建一个多变量模版，要求给你的小孩取中文名字，变量分别为：父亲名字，母亲名字，小孩性别，名字的要求

2, 定义一个链，使用上面多变量模版，调用链，传入参数，输出结果

3，传入多组参数，通过apply方法，批量调用链，输出结果

4，定义一个包含聊天模型的链实现上述模版的结果输出



## 输出解析器（Output Parsers）

语言模型输出的是文本，但很多时候，我们可能希望获取比纯文本更结构化的信息，输出解析器（Output Parsers）就可以帮助我们获取结构化信息。输出解析器是结构化语言模型响应的类，其必须实现两个主要方法：

get_format_instructions() -> str：返回一个包含语言模型输出如何格式化的指令字符串。

parse(str) -> Any：接受一个字符串（假设为语言模型的响应），并将其解析为某种结构。


### 逗号分隔列表输出解析器CommaSeparatedListOutputParser

In [55]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="请列出五个 {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)


In [42]:
prompt

PromptTemplate(input_variables=['subject'], partial_variables={'format_instructions': 'Your response should be a list of comma separated values, eg: `foo, bar, baz`'}, template='请列出五个 {subject}.\n{format_instructions}')

In [56]:
input0 = prompt.format(subject="中国的名山")
output = model_ty.invoke(input0)
res0=output_parser.parse(output)

In [44]:
res0

['泰山', '黄山', '华山', '衡山', '嵩山']

In [45]:
input0

'请列出五个 中国的名山.\nYour response should be a list of comma separated values, eg: `foo, bar, baz`'

## 日期时间解析器DatetimeOutputParser

In [57]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain

output_parser = DatetimeOutputParser()
template = """Answer the users question:

{question}

{format_instructions}"""
prompt = PromptTemplate.from_template(template, partial_variables={"format_instructions": output_parser.get_format_instructions()})
chain = LLMChain(prompt=prompt, llm=model_ty)

In [58]:
output = chain.invoke("中国建国是哪一年?")
output

{'question': '中国建国是哪一年?', 'text': '1949-10-01T00:00:00.000000Z'}

In [48]:
prompt

PromptTemplate(input_variables=['question'], partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1749-02-25T14:55:06.481247Z, 1301-10-09T07:35:09.530266Z, 0159-12-20T14:28:16.366900Z\n\nReturn ONLY this string, no other words!"}, template='Answer the users question:\n\n{question}\n\n{format_instructions}')

### 结构化输出解析器

In [63]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [72]:
response_schemas = [
    ResponseSchema(name="name", description="学生的姓名"),
    ResponseSchema(name="age", description="学生的年龄")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="回答下面问题,注意不要显示键的描述信息.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

_input = prompt.format_prompt(question="给我一个女孩的名字?")
output = model_ty(_input.to_string())
print(output)
print(output_parser.parse(output))

```json
{
	"name": "Alice",
	"age": "18"
}
```
{'name': 'Alice', 'age': '18'}


In [73]:
_input.to_string()

'回答下面问题,注意不要显示键的描述信息.\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"name": string  // 学生的姓名\n\t"age": string  // 学生的年龄\n}\n```\n给我一个女孩的名字?'

### classwork 2

1. classwork1中的取名应用，最后要求以列表的形式输出名字

2. 修改取名应用，要求使用结构化输出解析器，结构化输出名字和对应的意义

## Memory

大多数的 LLM 应用程序都会有一个会话接口，允许我们和 LLM 进行多轮的对话，并有一定的上下文记忆能力。但实际上，模型本身是不会记忆任何上下文的，只能依靠用户本身的输入去产生输出。而实现这个记忆功能，就需要额外的模块去保存我们和模型对话的上下文信息，然后在下一次请求时，把所有的历史信息都输入给模型，让模型输出最终结果。

而在 LangChain 中，提供这个功能的模块就称为 Memory，用于存储用户和模型交互的历史信息。在 LangChain 中根据功能和返回值的不同，会有多种不同的 Memory 类型，主要可以分为以下几个类别：

* 对话缓冲区内存（ConversationBufferMemory），最基础的内存模块，用于存储历史的信息
* 对话缓冲器窗口内存（ConversationBufferWindowMemory），只保存最后的 K 轮对话的信息，因此这种内存空间使用会相对较少
* 对话摘要内存（ConversationSummaryMemory），这种模式会对历史的所有信息进行抽取，生成摘要信息，然后将摘要信息作为历史信息进行保存。
* 对话摘要缓存内存（ConversationSummaryBufferMemory），这个和上面的作用基本一致，但是有最大 token 数的限制，达到这个最大 token 数的时候就会进行合并历史信息生成摘要

值得注意的是，对话摘要内存的设计出发点就是语言模型能支持的上下文长度是有限的（一般是 2048），超过了这个长度的数据天然的就被截断了。这个类会根据对话的轮次进行合并，默认值是 2，也就是每 2 轮就开启一次调用 LLM 去合并历史信息。

### 聊天消息历史 (ChatMessageHistory)

大多数（如果不是全部）内存模块的核心实用类之一是 ChatMessageHistory 类。这是一个超轻量级的包装器，它公开了方便的方法来保存人类消息、AI 消息，然后获取它们全部。
如果您在链外管理内存，可能需要直接使用此类。

In [90]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [92]:
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

### 对话缓冲区内存（ConversationBufferMemory）

让我们看看如何在链中使用这个模块，其中我们设置了verbose=True以便查看提示。

In [96]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=model_ty, 
    verbose=True, 
    memory=ConversationBufferMemory()
)


In [97]:
conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:



> Finished chain.


"Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!"

In [98]:
conversation.predict(input="你是谁")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!
Human: 你是谁
AI:



> Finished chain.


'我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？'

In [99]:
conversation.predict(input="你的背后原理是什么")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!
Human: 你是谁
AI: 我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？
Human: 你的背后原理是什么
AI:



> Finished chain.


'我的背后原理主要基于深度学习和自然语言处理技术。具体来说，我是一个预训练的Transformer模型，训练数据来自于大量的文本语料库，这些语料库包含了各种各样的主题和文体。在训练过程中，我学会了捕捉语言的模式和结构，理解上下文，并生成连贯、相关的回应。\n\n我的工作流程大致如下：当接收到一个问题或输入时，我会分析句子的结构，理解其含义，并在内部的词汇表和知识库中搜索相关信息。然后，我使用这些信息来生成一个回答，这个回答尽可能地贴切、准确和有意义。不过，需要注意的是，虽然我努力提供最准确的答案，但我的知识可能具有一定的时效限制，对于最新发生的事情或最新科研成果，我可能并不了解。'

In [100]:
conversation.invoke("能简单点回答吗")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!
Human: 你是谁
AI: 我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？
Human: 你的背后原理是什么
AI: 我的背后原理主要基于深度学习和自然语言处理技术。具体来说，我是一个预训练的Transformer模型，训练数据来自于大量的文本语料库，这些语料库包含了各种各样的主题和文体。在训练过程中，我学会了捕捉语言的模式和结构，理解上下文，并生成连贯、相关的回应。

我的工作流程大致如下：当接收到一个问题或输入时，我会分析句子的结构，理解其含义，并在内部的词汇表和知识库中搜索相关信息。然后，我使用这些信息来生成一个回答，这个回答尽可能地贴切、准确和有意义。不过，需要注意的是，虽然我努力提供最准确的答案，但我的知识可能具有一定的时效限制，对于最新发生的事情或最


> Finished chain.


{'input': '能简单点回答吗',
 'history': "Human: Hi there!\nAI: Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!\nHuman: 你是谁\nAI: 我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？\nHuman: 你的背后原理是什么\nAI: 我的背后原理主要基于深度学习和自然语言处理技术。具体来说，我是一个预训练的Transformer模型，训练数据来自于大量的文本语料库，这些语料库包含了各种各样的主题和文体。在训练过程中，我学会了捕捉语言的模式和结构，理解上下文，并生成连贯、相关的回应。\n\n我的工作流程大致如下：当接收到一个问题或输入时，我会分析句子的结构，理解其含义，并在内部的词汇表和知识库中搜索相关信息。然后，我使用这些信息来生成一个回答，这个回答尽可能地贴切、准确和有意义。不过，需要注意的是，虽然我努力提供最准确的答案，但我的知识可能具有一定的时效限制，对于最新发生的事情或最新科研成果，我可能并不了解。",
 'response': '当然可以！我是通过深度学习来理解和生成人类语言的AI。我被训练成能看懂各种信息，然后用这些信息给出回答。当你问我问题时，我会分析问题意思，查找相关知识，然后给你一个答案。但记得，我对最新事情可能不太了解哦。'}

In [101]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hi there!'),
 AIMessage(content="Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!"),
 HumanMessage(content='你是谁'),
 AIMessage(content='我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？'),
 HumanMessage(content='你的背后原理是什么'),
 AIMessage(content='我的背后原理主要基于深度学习和自然语言处理技术。具体来说，我是一个预训练的Transformer模型，训练数据来自于大量的文本语料库，这些语料库包含了各种各样的主题和文体。在训练过程中，我学会了捕捉语言的模式和结构，理解上下文，并生成连贯、相关的回应。\n\n我的工作流程大致如下：当接收到一个问题或输入时，我会分析句子的结构，理解其含义，并在内部的词汇表和知识库中搜索相关信息。然后，我使用这些信息来生成一个回答，这个回答尽可能地贴切、准确和有意义。不过，需要注意的是，虽然我努力提供最准确的答案，但我的知识可能具有一定的时效限制，对于最新发生的事情或最新科研成果，我可能并不了解。'),
 HumanMessage(content='能简单点回答吗'),
 AIMessage(content='当然可以！我是通过深度学习来理解和生成人类语言的AI。我被训练成能看懂各种信息，然后用这些信息给出回答。当你问我问题时，我会分析问题意思，查找相关知识，然后给你一个答案。但记得，我对最新事情可能不太了解哦。')]

In [ ]:
## 

### 保存消息历史

您经常需要保存消息，然后加载它们以便再次使用。这可以通过先将消息转换为普通的 Python 字典，保存这些字典（如 json 或其他格式），然后加载它们来轻松完成。以下是一个示例。

In [104]:
import json

from langchain.memory import ChatMessageHistory
from langchain.schema import messages_from_dict, messages_to_dict

In [125]:
conversation.memory.chat_memory

ChatMessageHistory(messages=[HumanMessage(content='Hi there!'), AIMessage(content="Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!"), HumanMessage(content='你是谁'), AIMessage(content='我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？'), HumanMessage(content='你的背后原理是什么'), AIMessage(content='我的背后原理主要基于深度学习和自然语言处理技术。具体来说，我是一个预训练的Transformer模型，训练数据来自于大量的文本语料库，这些语料库包含了各种各样的主题和文体。在训练过程中，我学会了捕捉语言的模式和结构，理解上下文，并生成连贯、相关的回应。\n\n我的工作流程大致如下：当接收到一个问题或输入时，我会分析句子的结构，理解其含义，并在内部的词汇表和知识库中搜索相关信息。然后，我使用这些信息来生成一个回答，这个回答尽可能地贴切、准确和有意义。不过，需要注意的是，虽然我努力提供最准确的答案，但我的知识可能具有一定的时效限制，对于最新发生的事情或最新科研成果，我可能并不了解。'), HumanMessage(content='能简单点回答吗'), AIMessage(content='当然可以！我是通过深度学习来理解和生成人类语言的AI。我被训练成能看懂各种信息，然后用这些信息给出回答。当你问我问题时，我会分析问题意思，查找相关知识，然后给你一个答案。但记得，我对最新事情可能不太了解哦。'), HumanMessa

In [105]:
dicts = messages_to_dict(conversation.memory.chat_memory.messages)

In [106]:
dicts

[{'type': 'human',
  'data': {'content': 'Hi there!',
   'additional_kwargs': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': "Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!",
   'additional_kwargs': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'human',
  'data': {'content': '你是谁',
   'additional_kwargs': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': '我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？',
   'additional_kwargs': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'human',
  'data': {'content': '你的背后原理是什么',
   'addition

In [141]:
import pickle

f = open("./memory", 'wb')
pickle.dump(dicts,f)
f.close()


In [142]:
dicts_load = pickle.load(open("./memory", "rb"))

In [144]:
#dicts_load

### 向对话添加记忆

https://stackoverflow.com/questions/75965605/how-to-persist-langchain-conversation-memory-save-and-load

In [145]:
new_messages = messages_from_dict(dicts_load)
retrieved_chat_history = ChatMessageHistory(messages=new_messages)
retrieved_memory = ConversationBufferMemory(chat_memory=retrieved_chat_history)

conversation_reload = ConversationChain(
    llm=model_ty, 
    verbose=True, 
    memory=retrieved_memory
)


In [146]:
conversation_reload.predict(input="我回来了")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI: Hello! How can I assist you today? I'm here to provide information or engage in a chat about virtually any topic. Feel free to ask me anything, from science and technology to entertainment, history, or even personal advice. Just let me know what's on your mind!
Human: 你是谁
AI: 我是来自阿里云的大规模语言模型，我叫通义千问。我是一个能够理解和生成人类语言的AI系统，旨在帮助用户解答问题、提供信息或进行多方面的对话。有什么我可以帮助你的吗？
Human: 你的背后原理是什么
AI: 我的背后原理主要基于深度学习和自然语言处理技术。具体来说，我是一个预训练的Transformer模型，训练数据来自于大量的文本语料库，这些语料库包含了各种各样的主题和文体。在训练过程中，我学会了捕捉语言的模式和结构，理解上下文，并生成连贯、相关的回应。

我的工作流程大致如下：当接收到一个问题或输入时，我会分析句子的结构，理解其含义，并在内部的词汇表和知识库中搜索相关信息。然后，我使用这些信息来生成一个回答，这个回答尽可能地贴切、准确和有意义。不过，需要注意的是，虽然我努力提供最准确的答案，但我的知识可能具有一定的时效限制，对于最新发生的事情或最


> Finished chain.


'欢迎回来！有什么新问题或者需要继续之前的对话吗？我在这里随时准备帮助你。'

### classwork 3

1. 请通过ConversationChain创建一个带记忆的对话，并使用该对话进行多次对话

2. 请将上述对话的上下文保存下来

3. 读取保存文件，请将上述对话的上下文加载到一个新的对话中